In [1]:
import os
import pandas as pd
import numpy as np
from simulation.constants import *
from datetime import date

In [3]:
df = pd.read_csv(DATA_FOLDER / 'copenhagen_interactions_distance.csv', parse_dates=["datetime"])
df["distance"] = -df["distance"]
## Remove distances over 2m (rssi lower than -90 is a proxy for that)
df = df[df['distance'] > -90].reset_index(drop=True)

In [4]:
def group_by_time_diff(df):
    groups = (df.sort_values(["source","destination"])["datetime"].diff() !=  pd.Timedelta('5m')).cumsum()
    df["groups"] = groups
    return (df.groupby(["source","destination", "groups"])
            .agg(**{
                "duration": pd.NamedAgg(column='datetime', aggfunc=lambda x: x.count() * 5), 
                "datetime": pd.NamedAgg(column='datetime', aggfunc='min')
            })
            .reset_index()
            .drop(columns=["groups"])
           )

In [5]:
%time new_df = df.groupby(["source", "destination"]).apply(group_by_time_diff)

CPU times: user 17min 10s, sys: 13.1 s, total: 17min 23s
Wall time: 19min 22s


In [6]:
new_df.reset_index(drop=True).sort_values("datetime").to_csv(DATA_FOLDER / "copenhagen_interactions_duration.csv", index=False)